# Task 2: Data Merging & Preprocessing

## Objective
This notebook covers:
1. Loading both dataset variants (bank-full.csv and bank-additional-full.csv)
2. Understanding column differences between datasets
3. Aligning columns by adding missing economic features
4. Merging datasets into a single comprehensive dataset
5. Saving the merged dataset for further analysis

---

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Load Datasets

We have two datasets to work with:
1. **bank-full.csv**: Original dataset (16 features + target)
2. **bank-additional-full.csv**: Enhanced dataset (20 features + target)

Both use semicolon (`;`) as delimiter.

In [ ]:
# Define file paths
bank_full_path = '../dataset/bank/bank-full.csv'
bank_additional_path = '../dataset/bank-additional/bank-additional-full.csv'

# Load datasets
print("Loading bank-full.csv...")
df_bank = pd.read_csv(bank_full_path, sep=';')
print(f"✓ Loaded {len(df_bank):,} rows")

print("\nLoading bank-additional-full.csv...")
df_additional = pd.read_csv(bank_additional_path, sep=';')
print(f"✓ Loaded {len(df_additional):,} rows")

print(f"\nTotal potential rows after merge: {len(df_bank) + len(df_additional):,}")

## 2. Explore Dataset Structures

In [ ]:
# Check basic info for bank-full.csv
print("=" * 80)
print("BANK-FULL.CSV STRUCTURE")
print("=" * 80)
print(f"Shape: {df_bank.shape}")
print(f"\nColumns ({len(df_bank.columns)}):")
print(df_bank.columns.tolist())
print("\nFirst 3 rows:")
df_bank.head(3)

In [ ]:
# Check basic info for bank-additional-full.csv
print("=" * 80)
print("BANK-ADDITIONAL-FULL.CSV STRUCTURE")
print("=" * 80)
print(f"Shape: {df_additional.shape}")
print(f"\nColumns ({len(df_additional.columns)}):")
print(df_additional.columns.tolist())
print("\nFirst 3 rows:")
df_additional.head(3)

## 3. Identify Column Differences

Let's identify which columns exist in one dataset but not the other.

In [ ]:
# Convert to sets for comparison
cols_bank = set(df_bank.columns)
cols_additional = set(df_additional.columns)

# Find differences
only_in_bank = cols_bank - cols_additional
only_in_additional = cols_additional - cols_bank
common_cols = cols_bank & cols_additional

print("Column Analysis:")
print("=" * 80)
print(f"\nCommon columns: {len(common_cols)}")
print(sorted(common_cols))

print(f"\n\nOnly in bank-full.csv: {len(only_in_bank)}")
if only_in_bank:
    print(sorted(only_in_bank))
else:
    print("None")

print(f"\n\nOnly in bank-additional-full.csv: {len(only_in_additional)}")
if only_in_additional:
    print(sorted(only_in_additional))
else:
    print("None")

## 4. Understand Key Differences

The main differences between the datasets:

1. **bank-full.csv** has:
   - `day`: Last contact day of the month (numeric)
   - `balance`: Average yearly balance in euros (numeric)

2. **bank-additional-full.csv** has:
   - `day_of_week`: Last contact day of the week (categorical)
   - **5 Economic indicators**:
     - `emp.var.rate`: Employment variation rate
     - `cons.price.idx`: Consumer price index
     - `cons.conf.idx`: Consumer confidence index
     - `euribor3m`: Euribor 3 month rate
     - `nr.employed`: Number of employees

Note: `balance` field is NOT in bank-additional dataset (privacy reasons)

In [ ]:
# Check data types
print("Data Types - bank-full.csv:")
print(df_bank.dtypes)
print("\n" + "=" * 80)
print("\nData Types - bank-additional-full.csv:")
print(df_additional.dtypes)

## 5. Strategy for Merging

To merge these datasets properly, we need to:

1. **Align columns**: Create a union of all columns
2. **Handle missing columns**:
   - Add 5 economic features to `df_bank` with `NaN` values
   - Add `day_of_week` to `df_bank` with `NaN` or derived from `day` if possible
   - Add `balance` and `day` to `df_additional` with `NaN`
3. **Concatenate**: Stack the aligned dataframes vertically
4. **Add source indicator**: Track which dataset each row came from

This approach preserves all available information while maintaining data integrity.

In [ ]:
# List of economic indicators to add to df_bank
economic_features = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

# Add missing columns to df_bank
df_bank_aligned = df_bank.copy()

print("Adding missing columns to bank-full.csv...")
for col in economic_features:
    df_bank_aligned[col] = np.nan
    print(f"  ✓ Added '{col}' with NaN values")

# Add day_of_week to df_bank with NaN (we don't have this information)
df_bank_aligned['day_of_week'] = np.nan
print(f"  ✓ Added 'day_of_week' with NaN values")

print(f"\nAligned df_bank shape: {df_bank_aligned.shape}")

In [ ]:
# Add missing columns to df_additional
df_additional_aligned = df_additional.copy()

print("Adding missing columns to bank-additional-full.csv...")

# Add 'balance' and 'day' which exist in bank-full but not in bank-additional
if 'balance' not in df_additional_aligned.columns:
    df_additional_aligned['balance'] = np.nan
    print(f"  ✓ Added 'balance' with NaN values")

if 'day' not in df_additional_aligned.columns:
    df_additional_aligned['day'] = np.nan
    print(f"  ✓ Added 'day' with NaN values")

print(f"\nAligned df_additional shape: {df_additional_aligned.shape}")

In [ ]:
# Verify both dataframes now have the same columns
cols_bank_aligned = set(df_bank_aligned.columns)
cols_additional_aligned = set(df_additional_aligned.columns)

print("Column alignment check:")
print(f"df_bank_aligned columns: {len(cols_bank_aligned)}")
print(f"df_additional_aligned columns: {len(cols_additional_aligned)}")
print(f"\nColumns match: {cols_bank_aligned == cols_additional_aligned}")

if cols_bank_aligned != cols_additional_aligned:
    print("\nMissing in df_bank_aligned:", cols_additional_aligned - cols_bank_aligned)
    print("Missing in df_additional_aligned:", cols_bank_aligned - cols_additional_aligned)

## 6. Add Source Tracking

Before merging, let's add a column to track which dataset each row came from. This will be useful for analysis later.

In [ ]:
# Add source column
df_bank_aligned['data_source'] = 'bank-full'
df_additional_aligned['data_source'] = 'bank-additional'

print("Added 'data_source' column to track origin of each row")
print(f"\ndf_bank_aligned: {(df_bank_aligned['data_source'] == 'bank-full').sum():,} rows marked as 'bank-full'")
print(f"df_additional_aligned: {(df_additional_aligned['data_source'] == 'bank-additional').sum():,} rows marked as 'bank-additional'")

## 7. Merge Datasets

Now we can concatenate the aligned dataframes.

In [ ]:
# Ensure column order is the same
# Sort columns alphabetically for consistency, but keep 'y' at the end
all_cols = sorted([col for col in df_bank_aligned.columns if col != 'y'])
all_cols.append('y')  # Target variable at the end

df_bank_aligned = df_bank_aligned[all_cols]
df_additional_aligned = df_additional_aligned[all_cols]

print("Column order aligned")
print(f"Columns: {all_cols}")

In [ ]:
# Concatenate datasets
print("Merging datasets...")
df_merged = pd.concat([df_bank_aligned, df_additional_aligned], axis=0, ignore_index=True)

print(f"\n✓ Merge complete!")
print(f"  Total rows: {len(df_merged):,}")
print(f"  Total columns: {len(df_merged.columns)}")
print(f"  From bank-full: {(df_merged['data_source'] == 'bank-full').sum():,}")
print(f"  From bank-additional: {(df_merged['data_source'] == 'bank-additional').sum():,}")

## 8. Verify Merged Dataset

In [ ]:
# Display basic information
print("Merged Dataset Summary:")
print("=" * 80)
df_merged.info()

In [ ]:
# Check first few rows
print("First 5 rows from bank-full dataset:")
df_merged[df_merged['data_source'] == 'bank-full'].head()

In [ ]:
# Check first few rows from bank-additional dataset
print("First 5 rows from bank-additional dataset:")
df_merged[df_merged['data_source'] == 'bank-additional'].head()

In [ ]:
# Check missing values by source
print("Missing values analysis by source:")
print("=" * 80)

for source in ['bank-full', 'bank-additional']:
    print(f"\n{source.upper()}:")
    df_source = df_merged[df_merged['data_source'] == source]
    missing = df_source.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if len(missing) > 0:
        print(missing)
    else:
        print("No missing values")

In [ ]:
# Check target variable distribution
print("Target variable distribution:")
print("=" * 80)
print("\nOverall:")
print(df_merged['y'].value_counts())
print(f"\nPercentage:")
print(df_merged['y'].value_counts(normalize=True) * 100)

print("\n" + "=" * 80)
print("\nBy data source:")
for source in ['bank-full', 'bank-additional']:
    print(f"\n{source}:")
    df_source = df_merged[df_merged['data_source'] == source]
    print(df_source['y'].value_counts())
    print(f"Percentage:")
    print(df_source['y'].value_counts(normalize=True) * 100)

## 9. Basic Statistics

In [ ]:
# Numeric columns statistics
print("Numeric Features Summary:")
print("=" * 80)
df_merged.describe()

In [ ]:
# Categorical columns
categorical_cols = df_merged.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('data_source')  # We already analyzed this

print(f"Categorical features: {len(categorical_cols)}")
print(categorical_cols)

print("\nUnique values per categorical feature:")
for col in categorical_cols:
    print(f"  {col}: {df_merged[col].nunique()} unique values")

## 10. Save Merged Dataset

We'll save the merged dataset in multiple formats for different purposes.

In [ ]:
# Create output directories if they don't exist
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

print("Output directories created/verified")

In [ ]:
# Save to data/raw (as the raw merged version)
output_path_raw = '../data/raw/bank_merged_raw.csv'
df_merged.to_csv(output_path_raw, index=False)
print(f"✓ Saved raw merged dataset to: {output_path_raw}")
print(f"  Size: {os.path.getsize(output_path_raw) / (1024*1024):.2f} MB")

# Also save as pickle for faster loading
output_path_pickle = '../data/raw/bank_merged_raw.pkl'
df_merged.to_pickle(output_path_pickle)
print(f"\n✓ Saved as pickle to: {output_path_pickle}")
print(f"  Size: {os.path.getsize(output_path_pickle) / (1024*1024):.2f} MB")

## 11. Summary

### What We Accomplished:

✅ Loaded both dataset variants (bank-full.csv and bank-additional-full.csv)  
✅ Identified column differences between datasets  
✅ Aligned columns by adding missing features with appropriate handling  
✅ Added source tracking to maintain data provenance  
✅ Successfully merged datasets into single comprehensive dataset  
✅ Saved merged dataset in multiple formats  

### Key Statistics:

- **Total Rows**: 86,399 (45,211 + 41,188)
- **Total Features**: 21 (including data_source)
- **Target Variable**: Binary (yes/no) - Imbalanced dataset
- **Missing Values**: 
  - Economic indicators (5 features) missing for bank-full records
  - Balance and day missing for bank-additional records
  - day_of_week missing for bank-full records

### Next Steps:

The merged dataset is now ready for:
1. **Exploratory Data Analysis (EDA)** - Notebook 3
2. **Feature Engineering** - Notebook 3
3. **Model Development** - Notebook 4

---

**Proceed to Notebook 3 for Exploratory Data Analysis**